# <center> **Base de données (Films)**

## **Présentation**

Des fichiers csv contenant les données ont été générés lors de la phase précédente de scrapping, nous allons maintenant lire ces fichiers csv dans des dataFrame Pandas et nettoyer les données puis les insérer dans une base de données SQL.


## **Questions**

**Quelles données mettre en SQL et en NoSQL ?**<br>
SQL pour les données tabulaires, NoSQL pour les données non tabulaires ou non structurées.

**Générer automatiquement des IDs chaines de caractères ?**

**Les requêtes faites à partir de notre API doivent avoir accès uniquement à la base en lecture seule**


## **Sources**

**Neo4j pour créer des graphs ML**<br>
https://neo4j.com/docs/getting-started/appendix/tutorials/guide-import-relational-and-etl/


**Wikidata**<br>
https://query.wikidata.org/querybuilder/?uselang=fr<br>


## **Création du schéma relationnel**

![relation_schema](images/DB_diagram.png)

Online tool: https://drawdb.vercel.app/editor<br>
https://www.mocodo.net/<br>

L'utilitaire en ligne **drawdb** permet d'exporter un fichier .SQL contenant la description de notre base (tables avec les relations)

![export_SQL](images/export_SQL.png)

Après quelques modifications à ce fichier nous pouvons créer notre base de données ainsi que les tables à partir de l'invite de commandes.

<code>"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code>



Une difficulté rencontrée est de faire apparaître les films similaires à un film dans la base de données, en effet cette situation est une relation "Many to many" où la table se référence elle-même, c'est un cas fréquent dans les réseaux sociaux où un utilisateur va avoir des amis eux-mêmes utilisateurs.<br>
Deux situations sont observées : <br>
- les relations symétriques (l'utilisateur Alice est amis avec l'utilisateur Bob implique nécessairement que Bob est ami d'Alice),
- les relations asymétriques (Alice est amis avec Bob mais Bob n'est pas forcément ami avec Alice).

Dans notre situation la relation est .....????

Pour représenter cette relation ....





**About Self many-to-many relationship**: https://stackoverflow.com/questions/17128472/many-to-many-on-same-table

Relations n-n

Remarque :
La table "reviews" sert de table de jonction entre les films et les utilisateurs, en effet la relation movies-users est une relation n-n car un utilateur peut écrire des avis pour plusieurs films et un film possède des avis de plusieurs utilisateurs.




## **Création de la base de données**

On lance **MySQL Shell** puis on passe en mode **SQ** avec l'instruction <code>\sql</code>

On peut ensuite lancer la création de notre base de données et des tables en lançant le script "movies.sql"

<code> "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code><br>

Une fois la base 'movies' créée, on peut créer un connecteur sur la base.

In [16]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='admin', \
                              host = '127.0.0.1', database='movies')
cursor = cnx.cursor(buffered=True)

In [14]:
cnx.disconnect()

## **Remplissage de la base de données**

### **Imports**

In [1]:
%reset

In [17]:
import math
import copy
import time
import re
import uuid
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display

pd.set_option('display.max_rows', 10)
tqdm.pandas()

### **Tools**

In [28]:
def string_with_comma_to_list_of_strings(st):
    ''' Convert a string such as "['string1', 'string2' ...]" into
        a list of string ['string1', 'string2', ...]

        Return: A list of strings.

        Arg:
         - st: string with the value to split.
    '''
    if pd.isna(st):
        return []
    return [item.strip() for item in st.split(",") if len(item.strip()) > 1]

def duration_to_minutes(st):
    ''' Convert duration string into number of minutes (integer)
        1h 35min   ---->    95

        Return: Integer representing the number of minutes.
        Arg:
         - st: duration string to convert.
    '''
    if pd.isna(st) or st == '': return 0
    if 'h' in st:
        a, b = st.split('h')
        if 'min' in b:
            b = b.replace('min', '')
            return 60 * int(a) + int(b.strip())
        return 60 * int(a)
    assert 'min' in st
    st = st.replace('min', '')
    return 60 * int(st)            

def unique_values_of_columns(df_data, column):
    ''' return a list with unique values found in the column,
        values in the column are like : 'value1,value2,value3' ..... 
        so we split all values in each row of the column and stack them in a list.

        Return : A list of values.

        Args:
         - df_data: dataframe with the data to extract,
         - column: string with the name of the column we want to work with.     
    '''
    df = df_data[column]
    df.dropna(inplace = True)
    df = df.apply(string_with_comma_to_list_of_strings)
    df = df.apply(pd.Series).stack().reset_index(drop=True)
    return df.unique()

def convert_months_FR_to_EN(st):
    ''' Convert french months to english months

        Return: string with a month in english.

        Arg:
         - st: string with a french date.
    '''
    if pd.isna(st): return ''
    for month_FR, month_EN in zip(['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre'],
                                  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']):
        if month_FR in st:
            return st.replace(month_FR, month_EN)
    print('ERROR', st)
    return st


# ------------------------ #
#                          #
#    Filling SQL tables    #
#                          #
# ------------------------ #

def generate_ID():
    return str(uuid.uuid4())

def fill_in_table(lst, table_name, field_id, field, connector, cursor):
    ''' Fill in an SQL table from a list of values.
        For each value in the list 'lst' we generate an ID and insert into the table (ID, value).

        CAREFUL: To run only once for each table, otherwise : "ERROR Duplicate entry '0' for key 'table_name.PRIMARY'"

        Return: A dictionary mapping each value of the list to an ID newly generated {value1 : ID1, value2 : ID2, ...}

        Args:
         - lst: list of values to insert into the table,
         - table_name: string with the name of the table,
         - field_id: string with the ID of the value record inserted in the table,
         - field: string of the field in the table,
         - connector: MySQL connector connected to the relevant database,
         - cursor: MySQL cursor to execute SQL statements.
    '''
    dic_return = {}
    for item in lst:
        dic_return[item] = generate_ID()
        sql = f"INSERT INTO {table_name} ({field_id}, {field}) VALUES (%s, %s)"
        val = (dic_return[item], item)
        cursor.execute(sql, val)
    connector.commit()
    return dic_return

def fill_in_pivot_table(table_name, field, lst_values, movie_id, cursor):
    ''' Fill in pivot table with couple value such (item, movie_id)
        where item is a value of lst_values.

        Args:
         - table_name: string with name of the pivot_table to fill in,
         - field: string with the name of the field (category_id, actor_id ...),
         - lst_values: list of values of the field,
         - movie_id: id of the movie to be connected
         - cursor: MySQL cursor to execute SQL statements.
    '''
    for item in lst_values:
        sql = f"INSERT INTO {table_name} ({field}, movie_id) VALUES (%s, %s)"
        val = (item, movie_id)
        cursor.execute(sql, val)

### **Lecture des données à partir des CSV**

In [34]:
ds_categories = pd.read_csv('csv/categories.csv', delimiter = ',')
ds_categories = ds_categories[ds_categories.columns[0]]
ds_countries = pd.read_csv('csv/countries.csv', delimiter = ',')
ds_countries = ds_countries[ds_countries.columns[0]]

df_movies = pd.read_csv('csv/movies_year_1982.csv', delimiter = ',')
# df_movies = pd.read_csv('csv/movies_decade_80.csv', delimiter = ',')
#df_movies = pd.read_csv('csv/movies_year_1960_to_1990.csv', delimiter = ',')

print("Categories :", ds_categories.shape)
print("Countries :", ds_countries.shape)
print("movies :", df_movies.shape)

Categories : (37,)
Countries : (104,)
movies : (81, 15)


### **Remplissage des tables**

In [20]:
# ------------------------------ #
#    Fill in categories table    #
# ------------------------------ #
dict_category_id = fill_in_table(ds_categories.values.tolist(), 'categories', 'category_id', 'category', cnx, cursor)

# ------------------------------ #
#    Fill in countries table     #
# ------------------------------ #
dict_country_id = fill_in_table(ds_countries.values.tolist(), 'countries', 'country_id', 'country', cnx, cursor)

# ------------------------------ #
#    Fill in directors table     #
# ------------------------------ #
lst_directors = unique_values_of_columns(df_movies, 'directors')
dict_director_id = fill_in_table(lst_directors, 'directors', 'director_id', 'director_name', cnx, cursor)

# ------------------------------ #
#      Fill in actors table      #
# ------------------------------ #
lst_actors = unique_values_of_columns(df_movies, 'actors')
dict_actor_id = fill_in_table(lst_actors, 'actors', 'actor_id', 'actor_name', cnx, cursor)

# ------------------------------ #
#     Fill in composers table    #
# ------------------------------ #
lst_composers = unique_values_of_columns(df_movies, 'composers')
dict_composer_id = fill_in_table(lst_composers, 'composers', 'composer_id', 'composer_name', cnx, cursor)

### **Remplissage de la table des films**

In [35]:
# --------------------------------------------------------
#
#  Converting some columns of "df_movies" into the appropriate format:
#   - correct date format
#   - duration un minutes
#   - IDs of the categories
#   - IDs of the directors / actors / composers
#
# --------------------------------------------------------

df_movies_formatted = df_movies.copy()
df_movies_formatted['categories'] = df_movies_formatted['categories'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['categories'] = df_movies_formatted['categories'].apply(lambda lst : [dict_category_id[k] for k in lst])

df_movies_formatted['countries'] = df_movies_formatted['countries'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['countries'] = df_movies_formatted['countries'].apply(lambda lst : [dict_country_id[k] for k in lst])

df_movies_formatted['directors'] = df_movies_formatted['directors'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['directors'] = df_movies_formatted['directors'].apply(lambda lst : [dict_director_id[k] for k in lst])

df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(lambda lst : [dict_actor_id[k] for k in lst])

df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(lambda lst : [dict_composer_id[k] for k in lst])

df_movies_formatted['duration'] = df_movies_formatted['duration'].apply(duration_to_minutes)

df_movies_formatted['date'] = df_movies_formatted['date'].apply(convert_months_FR_to_EN)
df_movies_formatted['date'] = pd.to_datetime(df_movies_formatted['date'], format='mixed')

df_movies_formatted['notes'] = df_movies_formatted['notes'].apply(int)
df_movies_formatted['reviews'] = df_movies_formatted['reviews'].astype(int)
df_movies_formatted['star_rating'] = df_movies_formatted['star_rating'].apply(lambda x : float(x.replace(',', '.')))

In [ ]:
for movie in df_movies_formatted.itertuples():

    # ----------- #
    #    infos    #
    # ----------- #

    info_id = generate_ID()
    sql = "INSERT INTO infos (info_id, summary, url_thumbnail) VALUES (%s, %s, %s)"
    val = (info_id, 
           movie[12],  # summary 
           movie[13])  # url_thumbnail
    cursor.execute(sql, val)

    # ----------- #
    #    movies   #
    # ----------- #

    movie_id = generate_ID()

    sql = "INSERT INTO movies (movie_id, title, release_date, duration, nb_notes, \
                               nb_reviews, info_id, star_rating) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (movie_id, 
           movie[1], # title
           movie[2], # release_date
           movie[3], # duration
           movie[7], # nb_notes
           movie[8], # nb_reviews
           info_id,  
           movie[6]) # star_rating
    cursor.execute(sql, val)

    # -------------------- #
    #     Pivot tables     #   
    # -------------------- #
    #    category_movie    #
    #    country_movie     #
    #    director_movie    #
    #    actor_movie       #
    #    composer_movie    #
    # -------------------- #

    # Fill in pivot table: category_movie
    lst_categories = movie[4]
    fill_in_pivot_table('category_movie', 'category_id', lst_categories, movie_id, cursor)

    # Fill in pivot table: country_movie
    lst_countries = movie[5]
    fill_in_pivot_table('country_movie', 'country_id', lst_countries, movie_id, cursor)

    # Fill in pivot table: director_movie
    lst_directors = movie[9]
    fill_in_pivot_table('director_movie', 'director_id', lst_directors, movie_id, cursor)

    # Fill in pivot table: actor_movie
    lst_actors = movie[10]
    fill_in_pivot_table('actor_movie', 'actor_id', lst_actors, movie_id, cursor)

    # Fill in pivot table: composer_movie
    lst_composers = movie[11]
    fill_in_pivot_table('composer_movie', 'composer_id', lst_composers, movie_id, cursor)
        
cnx.commit()

## **Quelques requêtes SQL**

**Requête** : Afficher les films dont le titre commence par "A" ainsi que les acteurs de ces films. 

In [307]:
query = (" \
SELECT m.title, a.actor_name \
FROM movies AS m \
JOIN actor_movie AS am ON am.movie_id = m.movie_id \
JOIN actors AS a ON a.actor_id = am.actor_id \
WHERE m.title LIKE 'A%' \
ORDER BY a.actor_name;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['title', 'actor'])
df

,title,actor
0,A nous la victoire,Amidou
1,A nous la victoire,Anton Diffring
2,A nous la victoire,Arthur Brauss
3,A nous la victoire,Benoit Ferreux
4,A nous la victoire,Bobby Moore
...,...,...
19,A nous la victoire,Paul Van Himst
20,A nous la victoire,Pelé
21,A nous la victoire,Sylvester Stallone
22,A nous la victoire,Tim Pigott-Smith


**Requête** : Afficher les films dont le star rating est supérieur à 4.5. 

In [314]:
query = (" \
SELECT m.title \
FROM movies AS m \
WHERE m.star_rating > 3.5;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['title'])
df

,title
0,Métal hurlant
1,Les Uns et les autres
2,Le Choix des armes
3,Blow Out
4,Reds
...,...
28,La Femme d'à côté
29,Le Choc des titans
30,Mad Max 2
31,Coup de Torchon


**Requête** : Afficher les films dont le star rating est supérieur à 4.5. 

In [320]:
query = (" \
SELECT a.actor_name \
FROM movies AS m \
JOIN actor_movie AS am ON am.movie_id = m.movie_id \
JOIN actors AS a ON a.actor_id = am.actor_id \
ORDER BY a.actor_name;")
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['actor'])
df

,actor
0,Abel Ferrara
1,Adrian Hoven
2,Adrienne Barbeau
3,Adrienne Barbeau
4,Adrienne King
...,...
1039,Yves-Marie Maurin
1040,Zack Norman
1041,Zoë Lund
1042,Zoltan Gera
